# <span style="color:purple">ArcGIS API for Python</span>


## Demo: Administrative Management of Users, Roles & Groups

### We will go over a few workflows to show how the ArcGIS API can help administrators of Web GIS.

Let's import a few modules. We are importing getpass to provide the GIS object with a password in a safe manner.

In [27]:
from arcgis.gis import *
from getpass import getpass
from IPython.display import display

Let's call our GIS object and pass a local Portal.

In [28]:
password = getpass()
gis = GIS("https://statesales.maps.arcgis.com", "pgamberg_statesales", password, verify_cert=False)

········


In [29]:
gis

GIS @ https://statesales.maps.arcgis.com version:7.2

### Querying for users

Let's run a full query for listed users:

In [4]:
gis.users.search()


[<User username:alis4396@esri.com_statesales>,
 <User username:ambe9440@esri.com_statesales>,
 <User username:anth5900@esri.com_statesales>,
 <User username:bbowles_statesales>,
 <User username:bjones_statesales>,
 <User username:bruc6988@esri.com_statesales>,
 <User username:bschmitz_statesales>,
 <User username:canderson_statesales>,
 <User username:cdorn_statesales>,
 <User username:cfisher_statesales>,
 <User username:chri1175@esri.com_statesales>,
 <User username:chri1610@esri.com_statesales>,
 <User username:chri4096@esri.com_statesales>,
 <User username:cnickola_statesales>,
 <User username:dan1988@esri.com_statesales>,
 <User username:davi9630@esri.com_statesales>,
 <User username:dhaag_esri_state>,
 <User username:eric3918@esri.com_statesales>,
 <User username:esristateadmin>,
 <User username:EsriStateSales>,
 <User username:jbowles_statesales>,
 <User username:jeckmann@esri.com_statesales>,
 <User username:jeckmann_statesales>,
 <User username:jeff2970@esri.com_statesales>,
 

In [30]:
gis.users.me

<User username:pgamberg_statesales>

Filter your user search...

In [6]:
gis.users.search("pete")

[<User username:pete9378@esri.com_statesales>,
 <User username:pgamberg_statesales>]

Let's search for my colleague, Matt

In [7]:
gis.users.search("Matt")

[<User username:matt3220@esri.com_statesales>,
 <User username:matt7399@esri.com_statesales>,
 <User username:Matt8683@esri.com_statesales>,
 <User username:matt8819@esri.com_statesales>,
 <User username:matt9602@esri.com_statesales>,
 <User username:mbullock_statesales>,
 <User username:mdenuelle_statesales>,
 <User username:MRobinson_statesales>,
 <User username:mviverito_statesales>]

### Create a new user

Users can be created through the API

In [ ]:
gis.users.create?

In [8]:
new_user = gis.users.create(username = "SmithCollinss", 
                           password="COLL@123", 
                          firstname="smith", 
                          lastname="colins", 
                          email="smith.collins@company.com",  
                          role='org_user')
new_user

<User username:SmithCollinss>

This variable is now a "user" object

In [9]:
type(new_user)

arcgis.gis.User

Use intellisense to see all the options you have available with user objects.

In [10]:
new_user.description

In [11]:
new_user.email

'smith.collins@company.com'

In [12]:
new_user.delete()

True

### Querying for groups

In [ ]:
gis.groups.search()

Let's create the group

In [13]:
# create groups
demo_group = gis.groups.create("Field Work", "Demo; CO")

This is a "group" object

In [31]:
demo_group?

In [15]:
demo_group

<Group title:"Field Work" owner:pgamberg_statesales>

In [16]:
type(demo_group)

arcgis.gis.Group

Like the user object, it has methods we can call on it

In [17]:
demo_group.access

'public'

# Let's say your boss suddenly gets an idea!


## Boss:
## "I want you to create a new portal, with new users from our regional teams and mulitple groups and make every member is part of their groups. 
## "And I need this before you go home."


## Have no fear! The ArcGIS API for Python is here to help.

We're going to import some libraries to help us

In [18]:
import csv 
import json

# Create Groups

In [19]:
def create_groups():
    print("CREATING GROUPS")

    with open("groups.csv", 'r') as groups_csv:
        groups = csv.DictReader(groups_csv)
        for group in groups:
            try:
                print(" Creating group: ", group['title'], end="  ##  ")
                result = gis.groups.create_from_dict(group)
                if result:
                    print("success")
            except Exception as create_ex:
                print("Error... ", str(create_ex))

# Add Users

In [20]:
def create_users():
    # loop through and create users
    with open("users.csv", 'r') as users_csv:
        users = csv.DictReader(users_csv)
        for user in users:
            try:
                print("Creating user: ", user['username'], end=" ## ")
                
                result = gis.users.create(username=user['username'],
                                          password=user['password'],
                                          firstname=user['Firstname'],
                                          lastname=user['Lastname'],
                                          email=user['email'],
                                          role=user['role'],
                                         )
                
                if result:
                    print("success  ##")

                    print("\t Adding to groups: ", end=" # ")
                    groups = user['groups']
                    group_list = groups.split(",")

                    # Search for the group
                    for g in group_list:
                        group_search = gis.groups.search(g)
                        if len(group_search) > 0:
                            try:
                                group = group_search[0]
                                groups_result = group.add_users([user['username']])
                                if len(groups_result['notAdded']) == 0:
                                    print(g, end =" # ")

                            except Exception as groups_ex:
                                print("\n \t Cannot add user to group ", g, str(groups_ex))
                    print("\n")

            except Exception as add_ex:
                print("Cannot create user: " + user['username'])
                print(str(add_ex))

# Publish Content

In [21]:
def publish_content():
    # Read the csv containing user accounts and their territory info
    csv_path = "users.csv"

    
    # Read template web map
    template_webmap_dict = dict()
    with open(".//user_content//web_map.json", 'r') as webmap_file:
                template_webmap_dict = json.load(webmap_file)

    # Loop through each user and publish the content
    with open(csv_path, 'r') as csv_handle:
        reader = csv.DictReader(csv_handle)
        for row in reader:
            try:
                data_to_publish = './/user_content//' + row['assigned_state'] + ".csv"

                print("Publishing ", data_to_publish, end = " # ")
                added_item = gis.content.add({}, data = data_to_publish)
                published_layer = added_item.publish()

                if published_layer is not None:
                    # publish web map
                    print('webmaps', end= " ## ")
                    user_webmap_dict = template_webmap_dict
                    user_webmap_dict['operationalLayers'][0].update({'itemId': published_layer.itemid,
                                                                     'layerType': "ArcGISFeatureLayer",
                                                                     'title': published_layer.title,
                                                                     'url': published_layer.url + r"/0"})

                    web_map_properties = {'title': '{0} {1} response locations'.format(row['Firstname'], row['Lastname']),
                                          'type': 'Web Map',
                                          'snippet': 'Areas affected by Hurricane Matthew under the supervision of' +\
                                                     '{0} {1}'.format(row['Firstname'], row['Lastname']),
                                          'tags': 'ArcGIS Python API',
                                          'typeKeywords' : "Collector, Explorer Web Map, Web Map, Map, Online Map",
                                          'text': json.dumps(user_webmap_dict)}

                    web_map_item = gis.content.add(web_map_properties)

                    print("success. Assigning to: ", end="  #  ")
                    result1 = published_layer.reassign_to(row['username'])
                    result2 = web_map_item.reassign_to(row['username'])
                    if (result1 and result2) is not None:
                        print(row['username'])
                    else:
                        print("error")
                else:
                    print(" error publishing csv")

            except Exception as pub_ex:
                print("Error : ", str(pub_ex))

# Customize the UX

In [22]:
def set_theme():
    
    folder = r'helper_files'
    print(folder)
    gis.admin.ux.set_logo(folder + r'\logo\dc_logo.png')
    gis.admin.ux.set_background(is_built_in=False)
    gis.admin.ux.set_banner(folder + r'\baked_banner\washingtondc_banner.png')
    gis.admin.ux.name = "Washington DC"
    gis.admin.ux.description = "Washington DC"

In [25]:
#set_theme()
create_groups()
create_users()
publish_content()

CREATING GROUPS
 Creating group:  Basemaps  ##  success
 Creating group:  Central Services  ##  success
 Creating group:  Compliance  ##  success
 Creating group:  Customer Service, Finance, Billing and Accounting  ##  success
 Creating group:  Demographic Content  ##  success
Creating user:  SmithCollins ## success  ##
	 Adding to groups:  # Basemaps #  Central Services # 

Creating user:  JohnsonStewart ## success  ##
	 Adding to groups:  # Basemaps #  Central Services # 

Creating user:  WilliamsSanchez ## success  ##
	 Adding to groups:  # Basemaps #  Customer Service # 

Publishing  .//user_content//KS.csv # webmaps ## success. Assigning to:   #  SmithCollins
Publishing  .//user_content//NV.csv # webmaps ## success. Assigning to:   #  JohnsonStewart
Publishing  .//user_content//IN.csv # webmaps ## success. Assigning to:   #  WilliamsSanchez


## "We just found out that we have to remove every user from our Portal as we have to use it for another urgent project.


## Good thing we have the ArcGIS API for Python to help.

We're going to run a clean up script(clean.py) in my local IDE(PyCharm) removing everything except the admin account and some groups. 



In [26]:
def clean():
    with open("users.csv", 'r') as users_csv:
        users = csv.DictReader(users_csv)
        for user in users:
            try:
                n = gis.users.get(user['username'])
                i = (n.items())
                for c in i:
                    c.delete()
                n.delete()
                
                

            except Exception as add_ex:
                print("exception " + user['username'])
                print(str(add_ex))
clean()